In [1]:
import os
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)
print('Version:',tf.__version__)

1 Physical GPUs, 1 Logical GPUs
Version: 2.3.1


In [3]:
model = tf.keras.models.load_model('Models/cat_dog_augmentation')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [4]:
from imutils.video import VideoStream
import imutils
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

In [5]:
def get_label(frame):
    x = image.img_to_array(frame)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    if classes[0]>0.5:
        label = "Cat"
    else:
        label = "Dog"
    return label

In [6]:
vs=VideoStream(src=0).start()

while True:
    frame=vs.read()
    frame=cv2.resize(frame,(150,150))
    
    label = get_label(frame)
    color=(0,255,0) if label=='Cat' else (0,0,255)
    cv2.putText(frame,label,(25,25),cv2.FONT_HERSHEY_SIMPLEX,0.75,color,2)

    #show the output frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    
    if key==ord('q'):
        break
cv2.destroyAllWindows()
vs.stop()
vs.stream.release()